# Import necessary libraries

In [1]:
import os #menejemenisasi file dan folder
# import cv2 
import numpy as np
from sklearn.utils import shuffle
from PIL import Image #library clean data
#library untuk menampilkan gambar
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#library tensorflow untuk pelatihan model 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #library untuk augmentasi gambar
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


# List all data in each classes

In [2]:
#Direktori data
base_dir = 'dataset'
#Direktori data daun sakit
blast_dir = os.path.join(base_dir, 'full/LeafBlast')
#Direktori data daun sehat
brownspot_dir = os.path.join(base_dir, 'full/BrownSpot')
#Direktori data daun sehat
hispa_dir = os.path.join(base_dir, 'full/Hispa')
healthy_dir = os.path.join(base_dir, 'full/Healthy')
#Mengambil semua anama file dalam masing-masing direktori
balst_fnames = os.listdir(blast_dir )
brownspot_fnames = os.listdir(brownspot_dir)
hispa_fnames = os.listdir(hispa_dir)
healthy_fnames = os.listdir(healthy_dir)

In [3]:
#check file
print('total data Blast:',len(os.listdir(blast_dir)))
print('total data Brownspot:',len(os.listdir(brownspot_dir)))
print('total data Hispa:',len(os.listdir(hispa_dir)))
print('total data healthy:',len(os.listdir(healthy_dir)))

total data Blast: 523
total data Brownspot: 523
total data Hispa: 523
total data healthy: 523


# Preview some samples 

In [4]:
# cols = 4
# rows = 3

# pic_index = 0
# #menampilkan gambar file
# img = plt.gcf()
# img.set_size_inches(cols*4, rows*3) 

# pic_index+=4

# #menyimpan ke dalam list nama file yang akan ditampilkan
# show_blast_img = [os.path.join(blast_dir, fname)
#                       for fname in balst_fnames[pic_index-4:pic_index] 
#                     ]

# show_brownspot_img = [os.path.join(brownspot_dir, fname)
#                       for fname in brownspot_fnames[pic_index-4:pic_index]
#                     ]
# show_hispa_img = [os.path.join(hispa_dir, fname)
#                       for fname in hispa_fnames[pic_index-4:pic_index]
#                     ]
# for i, img_path in enumerate(show_blast_img+show_brownspot_img+show_hispa_img):
#     sp = plt.subplot(rows, cols, i + 1)
#     sp.axis('off')

#     image = mpimg.imread(img_path)
#     plt.imshow(image)

# plt.show()

# Load Image using ImageDataGenerator

In [5]:
IMAGE_SHAPE = (200, 200)
DATA_DIR = os.path.join(base_dir, 'full')
#VALID_DATA_DIR = os.path.join(base_dir, 'test')

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, validation_split=0.2
)


train_generator = datagen.flow_from_directory(
    DATA_DIR,
    shuffle=False,
    subset='training',
    target_size=IMAGE_SHAPE,
    class_mode='categorical'
)
valid_generator = datagen.flow_from_directory(
    DATA_DIR,
    shuffle=False,
    subset='validation',
    target_size=IMAGE_SHAPE,
    class_mode='categorical'
)

Found 1676 images belonging to 4 classes.
Found 416 images belonging to 4 classes.


In [6]:
print(train_generator.class_indices)

{'BrownSpot': 0, 'Healthy': 1, 'Hispa': 2, 'LeafBlast': 3}


# Our model for classification

In [7]:
#create model
EPOCHS = 20
BATCH_SIZE = 32

NUM_CLASSES = 4

def save_plots(train_acc, valid_acc, train_loss, valid_loss):
    """
    Function to save the loss and accuracy plots to disk.
    """
    # accuracy plots
    plt.figure(figsize=(12, 9))
    plt.plot(
        train_acc, color='green', linestyle='-', 
        label='train accuracy'
    )
    plt.plot(
        valid_acc, color='blue', linestyle='-', 
        label='validataion accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    filepath = os.path.join(base_dir, 'figures')
    plt.savefig(filepath+'/accuracy.png')
    plt.show()
    # loss plots
    plt.figure(figsize=(12, 9))
    plt.plot(
        train_loss, color='orange', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-', 
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(filepath+'/loss.png')
    plt.show()

def get_model(train=False):
  
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape= (224,224,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation= 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128,(3,3), activation= 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128,(3,3), activation= 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),


        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation= 'relu'),
        tf.keras.layers.Dense(512, activation= 'relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(NUM_CLASSES, activation= 'softmax')
    ])

  #menampilkan summary/keterangan dari model
  

    model.compile(loss='categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])
    history = None
    filepath = os.path.join(base_dir, 'models')
    if train:
    
        checkpoint_path = filepath+"/training_1/cp.ckpt"
        checkpoint_dir = os.path.dirname(checkpoint_path)

        # Create a callback that saves the model's weights
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=1)


        history = model.fit(train_generator,
            steps_per_epoch=train_generator.samples // BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=valid_generator,
            callbacks=[cp_callback],
            validation_steps= valid_generator.samples // BATCH_SIZE,
            verbose=1
            )
        train_loss = history.history['loss']
        train_acc = history.history['accuracy']
        valid_loss = history.history['val_loss']
        valid_acc = history.history['val_accuracy']

        save_plots(train_acc, valid_acc, train_loss, valid_loss)
        model.save(save_format="h5",filepath=filepath+"/modelku")
    else:
        history = tf.keras.models.load_model(filepath+"/modelku")
    
    return history



# Training time

# Performance measurement

In [ ]:

model = get_model(train=True)

if model is not None:
    # model.summary()

    # test_labels = valid_generator.labels
    loss, acc = model.evaluate(valid_generator)
    print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


# print(valid_generator.labels)

2022-01-09 21:49:34.368586: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-01-09 21:49:34.474465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 21:49:34.475360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce GTX 1050 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.62
pciBusID: 0000:01:00.0
2022-01-09 21:49:34.475930: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2022-01-09 21:49:34.480099: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-01-09 21:49:34.483474: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library

2022-01-09 21:52:21.345857: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:143] Filling up shuffle buffer (this may take a while): 18 of 53
2022-01-09 21:52:32.359814: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:143] Filling up shuffle buffer (this may take a while): 19 of 53
2022-01-09 21:52:44.566020: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:143] Filling up shuffle buffer (this may take a while): 20 of 53
2022-01-09 21:52:58.643450: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:143] Filling up shuffle buffer (this may take a while): 21 of 53
2022-01-09 21:53:11.712035: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:143] Filling up shuffle buffer (this may take a while): 22 of 53
2022-01-09 21:53:25.302609: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:143] Filling up shuffle buffer (this may take a while): 23 of 53
2022-01-09 21:53:34.297075: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:143] Filling up shuffle buffer (this may ta